The dataset is the tweet archive of Twitter user _**@dog_rates, also known as WeRateDogs.**_ WeRateDogs is a Twitter account that rates people's dogs with a humorous comment about the dog. These ratings almost always have a denominator of 10. The numerators, though? Almost always greater than 10. 11/10, 12/10, 13/10, etc. Why? Because "they're good dogs Brent." WeRateDogs has over 4 million followers and has received international media coverage.

Gather each of the three pieces of data as described below in a Jupyter Notebook titled wrangle_act.ipynb:

1. The WeRateDogs Twitter archive. I am giving this file to you, so imagine it as a file on hand. Download this file manually by clicking the following link: _**twitter_archive_enhanced.csv**__

2. The tweet image predictions, i.e., what breed of dog (or other object, animal, etc.) is present in each tweet according to a neural network. This file (_**image_predictions.tsv**_) is hosted on Udacity's servers and should be downloaded programmatically using the **Requests** library and the following URL: https://d17h27t6h515a5.cloudfront.net/topher/2017/August/599fd2ad_image-predictions/image-predictions.tsv

3. _Each tweet's retweet count and favorite ("like")_ count at minimum, and any additional data you find interesting. Using the tweet IDs in the WeRateDogs Twitter archive, query the Twitter API for each tweet's JSON data using Python's Tweepy library and store each tweet's entire set of JSON data in a file called _**tweet_json.txt**_ file. Each tweet's JSON data should be written to its own line. Then read this .txt file line by line into a pandas DataFrame with (at minimum) tweet ID, retweet count, and favorite count. **Note: do not include your Twitter API keys, secrets, and tokens in your project submission.**

**Key points to keep in mind when data wrangling for this project:**

- You only want _**original ratings (no retweets) that have images**_. Though there are 5000+ tweets in the dataset, not all are dog ratings and some are retweets.
- Assessing and cleaning the entire dataset completely would require a lot of time, and is not necessary to practice and demonstrate your skills in data wrangling. Therefore, the requirements of this project are only to _**assess and clean at least 8 quality issues and at least 2 tidiness issues**_ in this dataset.
- Cleaning includes merging individual pieces of data according to the rules of tidy data.
- The fact that the rating numerators are greater than the denominators does not need to be cleaned. This unique rating system is a big part of the popularity of WeRateDogs.
- You do _**not need to gather the tweets beyond August 1st, 2017.**_ You can, but note that you won't be able to gather the image predictions for these tweets since you don't have access to the algorithm used.

**Storing, Analyzing, and Visualizing Data for this Project**
Store the clean DataFrame(s) in a CSV file with the main one named _**twitter_archive_master.csv.**_ If additional files exist because multiple tables are required for tidiness, name these files appropriately. Additionally, you may store the cleaned data in a _**SQLite database**_ (which is to be submitted as well if you do).

Analyze and visualize your wrangled data in your _**wrangle_act.ipynb**_ Jupyter Notebook. **At least three (3) insights and one (1) visualization must be produced.**

**Reporting for this Project**
Create a 300-600 word written report called _**wrangle_report.pdf**_ or _**wrangle_report.html**_ that briefly describes your wrangling efforts. This is to be framed as an internal document.

Create a 250-word-minimum written report called _**act_report.pdf or act_report.html**_ that communicates the insights and displays the visualization(s) produced from your wrangled data. This is to be framed as an external document, like a blog post or magazine article, for example.

Both of these documents can be created in separate Jupyter Notebooks using the Markdown functionality of Jupyter Notebooks, then downloading those notebooks as PDF files or HTML files (see image below). You might prefer to use a word processor like Google Docs or Microsoft Word, however.

## 1. Gather

In [6]:
import pandas as pd
import numpy as np
import tweepy
import os
import requests
import json
import collections
import functools
import matplotlib as plt
%matplotlib inline

### 1.1 Twitter archive enhanced

In [9]:
tw_archive = pd.read_csv('twitter-archive-enhanced.csv')

### 1.2 Image prediction file

In [4]:
# Obtain from url the image-predictions.tsv file

image_pred_url = 'https://d17h27t6h515a5.cloudfront.net/topher/2017/August/599fd2ad_image-predictions/image-predictions.tsv'

with open('image-predictions.tsv', 'wb') as image_pred_file:
    response = requests.get(image_pred_url)
    image_pred_file.write(response.content)

In [66]:
# Load image prediction data into a dataframe
tweet_image_pred = pd.read_csv('image-predictions.tsv', delim_whitespace=True)

### 1.3 Tweepy extra information

In [16]:
#tweepy API

consumer_key = 'xxx'
consumer_secret = 'xxx'
access_token = 'xxx'
access_secret = 'xxx'

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_secret)

api = tweepy.API(auth, parser=tweepy.parsers.JSONParser(), wait_on_rate_limit=True, wait_on_rate_limit_notify=True)

Also, note that the tweets corresponding to a few tweet IDs in the archive may have been deleted. _**Try-except blocks may come in handy here.**_

To query all of the tweet IDs in the WeRateDogs Twitter archive, 20-30 minutes of running time can be expected. _**Printing out each tweet ID after it was queried and using a code timer were both helpful for sanity reasons**_. Setting the _**wait_on_rate_limit**_ and _**wait_on_rate_limit_notify**_ parameters to _**True**_ in the tweepy.api class is useful as well.

In [17]:
tweet_ids = tw_archive.tweet_id.values

In [23]:
test_tweet = api.get_status(tweet_ids[0], tweet_mode='extended')
test_tweet

{'created_at': 'Tue Aug 01 16:23:56 +0000 2017',
 'id': 892420643555336193,
 'id_str': '892420643555336193',
 'full_text': "This is Phineas. He's a mystical boy. Only ever appears in the hole of a donut. 13/10 https://t.co/MgUWQ76dJU",
 'truncated': False,
 'display_text_range': [0, 85],
 'entities': {'hashtags': [],
  'symbols': [],
  'user_mentions': [],
  'urls': [],
  'media': [{'id': 892420639486877696,
    'id_str': '892420639486877696',
    'indices': [86, 109],
    'media_url': 'http://pbs.twimg.com/media/DGKD1-bXoAAIAUK.jpg',
    'media_url_https': 'https://pbs.twimg.com/media/DGKD1-bXoAAIAUK.jpg',
    'url': 'https://t.co/MgUWQ76dJU',
    'display_url': 'pic.twitter.com/MgUWQ76dJU',
    'expanded_url': 'https://twitter.com/dog_rates/status/892420643555336193/photo/1',
    'type': 'photo',
    'sizes': {'thumb': {'w': 150, 'h': 150, 'resize': 'crop'},
     'medium': {'w': 540, 'h': 528, 'resize': 'fit'},
     'small': {'w': 540, 'h': 528, 'resize': 'fit'},
     'large': {'w': 

In [24]:
len(tweet_ids)

2356

In [26]:
# Iterate over tweet_ids to obtain favorites and retweet count. Then append these has dicts into df_list.
df_list = []

# Any tweet_ids not found by API, are appended to e_list
e_list = []

for id in tweet_ids:
    try:
        page = api.get_status(id, tweet_mode = 'extended')
        favorites = page['favorite_count']
        retweet_count = page['retweet_count']
        df_list.append({'tweet_id': int(id),
                        'favorites': int(favorites),
                        'retweet_count': int(retweet_count)
                       })
        print(id, favorites, retweet_count)
    
    except Exception:
        e_list.append(id)

875021211251597312 23338 4156
874680097055178752 25437 4082
874434818259525634 0 12788
874296783580663808 23832 3638
874057562936811520 20794 3512
874012996292530176 31222 8997
873213775632977920 6551 1394
872967104147763200 24933 4770
872820683541237760 13515 3290
872620804844003328 19003 3217
872486979161796608 37348 8050
872122724285648897 31368 7173
871879754684805121 34364 9668
871762521631449091 18524 3081
871515927908634625 18500 3072
871102520638267392 19194 4887
871032628920680449 20873 3377
870804317367881728 30878 5480
870726314365509632 112 3
870656317836468226 11282 2308
870374049280663552 76497 23705
870308999962521604 20143 3685
870063196459192321 33550 7391
869772420881756160 39220 9055
869702957897576449 25930 5604
869596645499047938 14703 2768
869227993411051520 18786 3350
868880397819494401 60091 12832
868639477480148993 0 1836
868622495443632128 24481 4714
868552278524837888 9280 1836
867900495410671616 22346 3745
867774946302451713 31343 6556
867421006826221569 148

Rate limit reached. Sleeping for: 244


671182547775299584 1027 307


In [30]:
# Any tweet_ids not found by API, are appended to e_list_2
e_list_2 = []

for e in e_list:
    try:
        page = api.get_status(e, tweet_mode = 'extended')
        favorites = page['favorite_count']
        retweet_count = page['retweet_count']
        df_list.append({'tweet_id': int(e),
                        'favorites': int(favorites),
                        'retweet_count': int(retweet_count)
                       })
        print(e, favorites, retweet_count)
    
    except Exception:
        e_list_2.append(e)

892177421306343426 30336 5487
891815181378084864 22808 3629
891689557279858688 38307 7542
891327558926688256 36574 8128
891087950875897856 18446 2721
890971913173991426 10731 1760
890729181411237888 58979 16490
890609185150312448 25383 3770
890240255349198849 28956 6389
890006608113172480 27936 6414
889880896479866881 25397 4352
889665388333682689 43627 8745
889638837579907072 24518 3904
889531135344209921 13815 1969
889278841981685760 22880 4644
888917238123831296 26494 3916
888804989199671297 23252 3705
888554962724278272 17936 3014
888078434458587136 19835 3036
887705289381826560 27564 4716
887517139158093824 42166 10280
887473957103951883 62410 15707
887343217045368832 30604 9212
887101392804085760 27894 5211
886983233522544640 31590 6668
886736880519319552 10869 2775
886680336477933568 20462 3916
886366144734445568 19203 2763
886267009285017600 108 3
886258384151887873 25441 5526
886054160059072513 0 96
885984800019947520 29627 5866
885528943205470208 32696 5581
885518971528720385

Rate limit reached. Sleeping for: 562


676590572941893632 874 115


In [43]:
len(e_list_2), len(df_list), len(e_list_2) + len(df_list)

(476, 1880, 2356)

I've executed the iterative process of retrieveing the JSON files from twitter for the remaining 476 tweets (from e_list_2) but keep on receiving no response so I'll assume that these ones are not possible to obtain the information.

In [53]:
df_list[0:2]

[{'tweet_id': 875021211251597312, 'favorites': 23338, 'retweet_count': 4156},
 {'tweet_id': 874680097055178752, 'favorites': 25437, 'retweet_count': 4082}]

In [54]:
rt_like_df = pd.DataFrame(df_list, columns = ['tweet_id', 'favorites', 'retweet_count'])

rt_like_df.to_csv('tweet_json.txt', index=False)

In [60]:
tweepy_info = pd.read_csv('tweet_json.txt')

## 2. Assess

### 2.1 Visual Assessment

#### 2.1.1 Twitter archive enhanced

In [8]:
tw_archive

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
0,892420643555336193,NaN,NaN,2017-08-01 16:23:56 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Phineas. He's a mystical boy. Only eve...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892420643...,13,10,Phineas,None,None,None,None
1,892177421306343426,NaN,NaN,2017-08-01 00:17:27 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Tilly. She's just checking pup on you....,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892177421...,13,10,Tilly,None,None,None,None
2,891815181378084864,NaN,NaN,2017-07-31 00:18:03 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Archie. He is a rare Norwegian Pouncin...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891815181...,12,10,Archie,None,None,None,None
3,891689557279858688,NaN,NaN,2017-07-30 15:58:51 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Darla. She commenced a snooze mid meal...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891689557...,13,10,Darla,None,None,None,None
4,891327558926688256,NaN,NaN,2017-07-29 16:00:24 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Franklin. He would like you to stop ca...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891327558...,12,10,Franklin,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2351,666049248165822465,NaN,NaN,2015-11-16 00:24:50 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Here we have a 1949 1st generation vulpix. Enj...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/666049248...,5,10,None,None,None,None,None
2352,666044226329800704,NaN,NaN,2015-11-16 00:04:52 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is a purebred Piers Morgan. Loves to Netf...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/666044226...,6,10,a,None,None,None,None
2353,666033412701032449,NaN,NaN,2015-11-15 23:21:54 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Here is a very happy pup. Big fan of well-main...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/666033412...,9,10,a,None,None,None,None
2354,666029285002620928,NaN,NaN,2015-11-15 23:05:30 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is a western brown Mitsubishi terrier. Up...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/666029285...,7,10,a,None,None,None,None


**Quality** issues summary:
- HTML tags in source text.
- name columns contains words that are not names (ex. 'a').
- The missing data in "name" or type of dog columns is encoded as 'None'.

**Tidiness** issues summary:
- There are four columns indicating the type of dog in the tweets: doggo, floofer, pupper, and puppo. These columns can actually be melted into a single column.

#### 2.1.2 Image prediction file

In [9]:
tweet_image_pred

,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
0,666020888022790149,https://pbs.twimg.com/media/CT4udn0WwAA0aMy.jpg,1,Welsh_springer_spaniel,0.465074,True,collie,0.156665,True,Shetland_sheepdog,0.061428,True
1,666029285002620928,https://pbs.twimg.com/media/CT42GRgUYAA5iDo.jpg,1,redbone,0.506826,True,miniature_pinscher,0.074192,True,Rhodesian_ridgeback,0.072010,True
2,666033412701032449,https://pbs.twimg.com/media/CT4521TWwAEvMyu.jpg,1,German_shepherd,0.596461,True,malinois,0.138584,True,bloodhound,0.116197,True
3,666044226329800704,https://pbs.twimg.com/media/CT5Dr8HUEAA-lEu.jpg,1,Rhodesian_ridgeback,0.408143,True,redbone,0.360687,True,miniature_pinscher,0.222752,True
4,666049248165822465,https://pbs.twimg.com/media/CT5IQmsXIAAKY4A.jpg,1,miniature_pinscher,0.560311,True,Rottweiler,0.243682,True,Doberman,0.154629,True
...,...,...,...,...,...,...,...,...,...,...,...,...
2070,891327558926688256,https://pbs.twimg.com/media/DF6hr6BUMAAzZgT.jpg,2,basset,0.555712,True,English_springer,0.225770,True,German_short-haired_pointer,0.175219,True
2071,891689557279858688,https://pbs.twimg.com/media/DF_q7IAWsAEuuN8.jpg,1,paper_towel,0.170278,False,Labrador_retriever,0.168086,True,spatula,0.040836,False
2072,891815181378084864,https://pbs.twimg.com/media/DGBdLU1WsAANxJ9.jpg,1,Chihuahua,0.716012,True,malamute,0.078253,True,kelpie,0.031379,True
2073,892177421306343426,https://pbs.twimg.com/media/DGGmoV4XsAAUL6n.jpg,1,Chihuahua,0.323581,True,Pekinese,0.090647,True,papillon,0.068957,True


**Quality** issues summary:
- Dog breed predictions with "(\_)" separator.
- Some predictions are not dog breeds (ex. paper_towel or orange)

**Tidiness** issues summary:
- "True" dog breed predictions should be merged into the "tw_archive" to consolidate all the info together.

#### 2.1.3 Tweepy extra info

In [61]:
tweepy_info.head()

,tweet_id,favorites,retweet_count
0,875021211251597312,23338,4156
1,874680097055178752,25437,4082
2,874434818259525634,0,12788
3,874296783580663808,23832,3638
4,874057562936811520,20794,3512


**Tidiness** issues summary:
- Should be merged into the "tw_archive" to consolidate all the info together.

### 2.2 Programatic Assessment

#### 2.2.1 Twitter archive enhanced

In [10]:
tw_archive.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2356 entries, 0 to 2355
Data columns (total 17 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   tweet_id                    2356 non-null   int64  
 1   in_reply_to_status_id       78 non-null     float64
 2   in_reply_to_user_id         78 non-null     float64
 3   timestamp                   2356 non-null   object 
 4   source                      2356 non-null   object 
 5   text                        2356 non-null   object 
 6   retweeted_status_id         181 non-null    float64
 7   retweeted_status_user_id    181 non-null    float64
 8   retweeted_status_timestamp  181 non-null    object 
 9   expanded_urls               2297 non-null   object 
 10  rating_numerator            2356 non-null   int64  
 11  rating_denominator          2356 non-null   int64  
 12  name                        2356 non-null   object 
 13  doggo                       2356 

181 retweets as seen in retweeted_status_id

In [11]:
tw_archive.describe()

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,retweeted_status_id,retweeted_status_user_id,rating_numerator,rating_denominator
count,2.356000e+03,7.800000e+01,7.800000e+01,1.810000e+02,1.810000e+02,2356.000000,2356.000000
mean,7.427716e+17,7.455079e+17,2.014171e+16,7.720400e+17,1.241698e+16,13.126486,10.455433
std,6.856705e+16,7.582492e+16,1.252797e+17,6.236928e+16,9.599254e+16,45.876648,6.745237
min,6.660209e+17,6.658147e+17,1.185634e+07,6.661041e+17,7.832140e+05,0.000000,0.000000
25%,6.783989e+17,6.757419e+17,3.086374e+08,7.186315e+17,4.196984e+09,10.000000,10.000000
50%,7.196279e+17,7.038708e+17,4.196984e+09,7.804657e+17,4.196984e+09,11.000000,10.000000
75%,7.993373e+17,8.257804e+17,4.196984e+09,8.203146e+17,4.196984e+09,12.000000,10.000000
max,8.924206e+17,8.862664e+17,8.405479e+17,8.874740e+17,7.874618e+17,1776.000000,170.000000


In [12]:
#very high/low numerator and denominator values to review

text_denom = tw_archive[tw_archive.rating_denominator != 10].text.to_list()
text_denom

["@jonnysun @Lin_Manuel ok jomny I know you're excited but 960/00 isn't a valid rating, 13/10 is tho",
 '@docmisterio account started on 11/15/15',
 'The floofs have been released I repeat the floofs have been released. 84/70 https://t.co/NIYC820tmd',
 'Meet Sam. She smiles 24/7 &amp; secretly aspires to be a reindeer. \r\nKeep Sam smiling by clicking and sharing this link:\r\nhttps://t.co/98tB8y7y7t https://t.co/LouL5vdvxx',
 'RT @dog_rates: After so many requests, this is Bretagne. She was the last surviving 9/11 search dog, and our second ever 14/10. RIP https:/…',
 'Why does this never happen at my front door... 165/150 https://t.co/HmwrdfEfUE',
 'After so many requests, this is Bretagne. She was the last surviving 9/11 search dog, and our second ever 14/10. RIP https://t.co/XAVDNDaVgQ',
 'Say hello to this unbelievably well behaved squad of doggos. 204/170 would try to pet all at once https://t.co/yGQI3He3xv',
 'Happy 4/20 from the squad! 13/10 for all https://t.co/eV1diwds8a',
 '

In [13]:
tw_archive.loc[342]

tweet_id                                                     832088576586297345
in_reply_to_status_id                                      832087547559997440.0
in_reply_to_user_id                                                  30582082.0
timestamp                                             2017-02-16 04:45:50 +0000
source                        <a href="http://twitter.com/download/iphone" r...
text                                   @docmisterio account started on 11/15/15
retweeted_status_id                                                         NaN
retweeted_status_user_id                                                    NaN
retweeted_status_timestamp                                                  NaN
expanded_urls                                                               NaN
rating_numerator                                                             11
rating_denominator                                                           15
name                                    

As seen in the texts above, the ratings with denominator != 10 are because:
- were incorrectly handled when colleting information as appear two ratings and the first one was taken
- some ratings do not follow rule of denominator 10 (ex. rating 80/80, 143/130, etc)
- second tweet from the list is showing a date ("account starte on..") and no information.

In [14]:
tw_archive.rating_denominator.value_counts(), tw_archive.rating_numerator.value_counts()

(10     2333
 11        3
 50        3
 20        2
 80        2
 0         1
 120       1
 7         1
 170       1
 150       1
 130       1
 90        1
 110       1
 2         1
 70        1
 40        1
 16        1
 15        1
 Name: rating_denominator, dtype: int64,
 12      558
 11      464
 10      461
 13      351
 9       158
 8       102
 7        55
 14       54
 5        37
 6        32
 3        19
 4        17
 1         9
 2         9
 0         2
 15        2
 75        2
 420       2
 182       1
 204       1
 143       1
 121       1
 99        1
 20        1
 45        1
 27        1
 17        1
 24        1
 26        1
 44        1
 50        1
 60        1
 80        1
 84        1
 88        1
 1776      1
 960       1
 666       1
 144       1
 165       1
 Name: rating_numerator, dtype: int64)

We can see that the outliers both for numerator(<10 and estremely high values) and denominator (=! 10) are not frequent compared to the dataset

In [15]:
tw_archive.duplicated().sum()

0

In [16]:
tw_archive.tweet_id.duplicated().sum()

0

**Quality** issues summary:
- Incorrect datatypes (*_id, *timestamp)
- retweet information not needed for this project
- some rating numerator and denominator have extreme max/min values
- some tweets contain no information

#### 2.2.2 Image prediction file

In [17]:
tweet_image_pred.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2075 entries, 0 to 2074
Data columns (total 12 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   tweet_id  2075 non-null   int64  
 1   jpg_url   2075 non-null   object 
 2   img_num   2075 non-null   int64  
 3   p1        2075 non-null   object 
 4   p1_conf   2075 non-null   float64
 5   p1_dog    2075 non-null   bool   
 6   p2        2075 non-null   object 
 7   p2_conf   2075 non-null   float64
 8   p2_dog    2075 non-null   bool   
 9   p3        2075 non-null   object 
 10  p3_conf   2075 non-null   float64
 11  p3_dog    2075 non-null   bool   
dtypes: bool(3), float64(3), int64(2), object(4)
memory usage: 152.1+ KB


In [18]:
tweet_image_pred.describe()

,tweet_id,img_num,p1_conf,p2_conf,p3_conf
count,2.075000e+03,2075.000000,2075.000000,2.075000e+03,2.075000e+03
mean,7.384514e+17,1.203855,0.594548,1.345886e-01,6.032417e-02
std,6.785203e+16,0.561875,0.271174,1.006657e-01,5.090593e-02
min,6.660209e+17,1.000000,0.044333,1.011300e-08,1.740170e-10
25%,6.764835e+17,1.000000,0.364412,5.388625e-02,1.622240e-02
50%,7.119988e+17,1.000000,0.588230,1.181810e-01,4.944380e-02
75%,7.932034e+17,1.000000,0.843855,1.955655e-01,9.180755e-02
max,8.924206e+17,4.000000,1.000000,4.880140e-01,2.734190e-01


The values seem correct as the predictions % are between [0-1]

In [19]:
tweet_image_pred.p1.value_counts()

golden_retriever      150
Labrador_retriever    100
Pembroke               89
Chihuahua              83
pug                    57
                     ... 
crash_helmet            1
stove                   1
giant_panda             1
lorikeet                1
rain_barrel             1
Name: p1, Length: 378, dtype: int64

In [20]:
tweet_image_pred.duplicated().sum()

0

In [21]:
tweet_image_pred.tweet_id.duplicated().sum()

0

**Quality** issues summary:
- Incorrect datatypes (*_id)
- Missing information as "tw_archive" file has 2356 rows and this one 2075. Nothing can be done as the predictions file is given and the neural net is not accessible.

#### 2.2.3 Tweepy extra information

In [63]:
tweepy_info.describe()

,tweet_id,favorites,retweet_count
count,1.880000e+03,1880.000000,1880.000000
mean,7.588648e+17,8652.038830,3045.656383
std,6.524954e+16,12034.913444,4670.826023
min,6.711825e+17,0.000000,1.000000
25%,6.972677e+17,2176.500000,797.750000
50%,7.504678e+17,4264.000000,1666.000000
75%,8.142473e+17,10901.250000,3463.500000
max,8.921774e+17,150577.000000,74317.000000


In [64]:
tweepy_info.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1880 entries, 0 to 1879
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype
---  ------         --------------  -----
 0   tweet_id       1880 non-null   int64
 1   favorites      1880 non-null   int64
 2   retweet_count  1880 non-null   int64
dtypes: int64(3)
memory usage: 44.2 KB


There doesn't seem to be any issue for this dataset on quality or tidiness. Nevertheless more than 400 tweets were not retreivable through tweepy API.

### Summary of quality and tidiness issues

**Quality**
1. <u>Twitter archive enhanced
    - HTML tags in source text.
    - name columns contains words that are not names (ex. 'a').
    - The missing data in "name" or type of dog columns is encoded as 'None'.
    - Incorrect datatypes (*_id, *timestamp)
    - retweet information not needed for this project
    - some rating numerator and denominator have extreme max/min values
    - some tweets contain no information

2. <u>Image prediction file:
    - Dog breed predictions with "(_)" separator.
    - Some predictions are not dog breeds (ex. paper_towel or orange)
    - Incorrect datatypes (*_id)
    - Missing information as "tw_archive" file has 2356 rows and this one 2075. Nothing can be done as the predictions file is given and the neural net is not accessible.

3. <u>Tweepy extra info
    - -

**Tidiness**
1. <u>Twiter archive enhanced:
    - There are four columns indicating the type of dog in the tweets: doggo, floofer, pupper, and puppo. These columns can actually be melted into a single column.

2. <u>Image prediction file:
    - "True" dog breed predictions should be merged into the "tw_archive" to consolidate all the info together.

3. <u>Tweepy extra info:
    - retweet_count and favorite_count should be part of the consolidated dataset.

## Clean

In [72]:
# Create copies of the tweet_data_archived, tweet_data_extra and tweet_image_pred dataframes

tw_archive_clean = tw_archive.copy()
pred_clean = tweet_image_pred.copy()
tweepy_clean = tweepy_info.copy()

### <u>Tidiness Issue 1 = 'There are four columns: doggo, floofer, pupper, and puppo'<u>

#### __*Define*__

Melt the 4 different columns of the dog types into one single column showing the dog type per row

#### __*Code*__

In [73]:
tw_archive_clean.doggo.value_counts(), tw_archive_clean.floofer.value_counts(), tw_archive_clean.pupper.value_counts(), tw_archive_clean.puppo.value_counts()

(None     2259
 doggo      97
 Name: doggo, dtype: int64,
 None       2346
 floofer      10
 Name: floofer, dtype: int64,
 None      2099
 pupper     257
 Name: pupper, dtype: int64,
 None     2326
 puppo      30
 Name: puppo, dtype: int64)

In [74]:
tw_archive_clean = tw_archive_clean.melt(id_vars=['tweet_id', 'in_reply_to_status_id', 'in_reply_to_user_id', 'timestamp', 'source', 'text', 'retweeted_status_id', 'retweeted_status_user_id', 'retweeted_status_timestamp', 'expanded_urls', 'rating_numerator',
'rating_denominator', 'name'], value_name= 'dog_type', value_vars=['doggo', 'floofer', 'pupper','puppo'])

In [75]:
tw_archive_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9424 entries, 0 to 9423
Data columns (total 15 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   tweet_id                    9424 non-null   int64  
 1   in_reply_to_status_id       312 non-null    float64
 2   in_reply_to_user_id         312 non-null    float64
 3   timestamp                   9424 non-null   object 
 4   source                      9424 non-null   object 
 5   text                        9424 non-null   object 
 6   retweeted_status_id         724 non-null    float64
 7   retweeted_status_user_id    724 non-null    float64
 8   retweeted_status_timestamp  724 non-null    object 
 9   expanded_urls               9188 non-null   object 
 10  rating_numerator            9424 non-null   int64  
 11  rating_denominator          9424 non-null   int64  
 12  name                        9424 non-null   object 
 13  variable                    9424 

we can see that now the dataframe has 2356x4=9424 rows as has duplicated the dog_types.

In [76]:
tw_archive_clean.dog_type.value_counts()

None       9030
pupper      257
doggo        97
puppo        30
floofer      10
Name: dog_type, dtype: int64

In [77]:
tw_archive_clean = tw_archive_clean.sort_values('dog_type').drop_duplicates('tweet_id', keep = 'last')

In [78]:
tw_archive_clean = tw_archive_clean.drop('variable', axis=1)

#### _**Test**_

In [79]:
tw_archive_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2356 entries, 2259 to 7236
Data columns (total 14 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   tweet_id                    2356 non-null   int64  
 1   in_reply_to_status_id       78 non-null     float64
 2   in_reply_to_user_id         78 non-null     float64
 3   timestamp                   2356 non-null   object 
 4   source                      2356 non-null   object 
 5   text                        2356 non-null   object 
 6   retweeted_status_id         181 non-null    float64
 7   retweeted_status_user_id    181 non-null    float64
 8   retweeted_status_timestamp  181 non-null    object 
 9   expanded_urls               2297 non-null   object 
 10  rating_numerator            2356 non-null   int64  
 11  rating_denominator          2356 non-null   int64  
 12  name                        2356 non-null   object 
 13  dog_type                    23

In [80]:
tw_archive_clean.dog_type.value_counts()

None       1976
pupper      257
doggo        83
puppo        30
floofer      10
Name: dog_type, dtype: int64

Now there are 14 values less for doggo type. This is due to rows where there was more than one dog_type informed

### <u>Tidiness Issue 2: 'tweet_image_pred: correct predictions should be combined with tweet_data_archived'<u>

#### __*Define*__

Extract the most likely "True" prediction from "tweet_image_pred" and merge them to the "tw_archive" file

#### __*Code*__

In [81]:
pred_clean

,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
0,666020888022790149,https://pbs.twimg.com/media/CT4udn0WwAA0aMy.jpg,1,Welsh_springer_spaniel,0.465074,True,collie,0.156665,True,Shetland_sheepdog,0.061428,True
1,666029285002620928,https://pbs.twimg.com/media/CT42GRgUYAA5iDo.jpg,1,redbone,0.506826,True,miniature_pinscher,0.074192,True,Rhodesian_ridgeback,0.072010,True
2,666033412701032449,https://pbs.twimg.com/media/CT4521TWwAEvMyu.jpg,1,German_shepherd,0.596461,True,malinois,0.138584,True,bloodhound,0.116197,True
3,666044226329800704,https://pbs.twimg.com/media/CT5Dr8HUEAA-lEu.jpg,1,Rhodesian_ridgeback,0.408143,True,redbone,0.360687,True,miniature_pinscher,0.222752,True
4,666049248165822465,https://pbs.twimg.com/media/CT5IQmsXIAAKY4A.jpg,1,miniature_pinscher,0.560311,True,Rottweiler,0.243682,True,Doberman,0.154629,True
...,...,...,...,...,...,...,...,...,...,...,...,...
2070,891327558926688256,https://pbs.twimg.com/media/DF6hr6BUMAAzZgT.jpg,2,basset,0.555712,True,English_springer,0.225770,True,German_short-haired_pointer,0.175219,True
2071,891689557279858688,https://pbs.twimg.com/media/DF_q7IAWsAEuuN8.jpg,1,paper_towel,0.170278,False,Labrador_retriever,0.168086,True,spatula,0.040836,False
2072,891815181378084864,https://pbs.twimg.com/media/DGBdLU1WsAANxJ9.jpg,1,Chihuahua,0.716012,True,malamute,0.078253,True,kelpie,0.031379,True
2073,892177421306343426,https://pbs.twimg.com/media/DGGmoV4XsAAUL6n.jpg,1,Chihuahua,0.323581,True,Pekinese,0.090647,True,papillon,0.068957,True


In [82]:
breed = []
prob = []

def breed_prob(row):
    if row['p1_dog'] == True:
        breed.append(row['p1'])
        prob.append(row['p1_conf'])
    elif row['p2_dog'] == True:
        breed.append(row['p2'])
        prob.append(row['p2_conf'])
    elif row['p3_dog'] == True:
        breed.append(row['p3'])
        prob.append(row['p3_conf'])
    else:
        breed.append('Unknown')
        prob.append(0)
        
# apply by columns
pred_clean.apply(breed_prob, axis=1)

# add lists created to master dataframe
pred_clean['breed'] = breed
pred_clean['prob'] = prob

In [83]:
pred_clean.head()

,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog,breed,prob
0,666020888022790149,https://pbs.twimg.com/media/CT4udn0WwAA0aMy.jpg,1,Welsh_springer_spaniel,0.465074,True,collie,0.156665,True,Shetland_sheepdog,0.061428,True,Welsh_springer_spaniel,0.465074
1,666029285002620928,https://pbs.twimg.com/media/CT42GRgUYAA5iDo.jpg,1,redbone,0.506826,True,miniature_pinscher,0.074192,True,Rhodesian_ridgeback,0.072010,True,redbone,0.506826
2,666033412701032449,https://pbs.twimg.com/media/CT4521TWwAEvMyu.jpg,1,German_shepherd,0.596461,True,malinois,0.138584,True,bloodhound,0.116197,True,German_shepherd,0.596461
3,666044226329800704,https://pbs.twimg.com/media/CT5Dr8HUEAA-lEu.jpg,1,Rhodesian_ridgeback,0.408143,True,redbone,0.360687,True,miniature_pinscher,0.222752,True,Rhodesian_ridgeback,0.408143
4,666049248165822465,https://pbs.twimg.com/media/CT5IQmsXIAAKY4A.jpg,1,miniature_pinscher,0.560311,True,Rottweiler,0.243682,True,Doberman,0.154629,True,miniature_pinscher,0.560311


In [84]:
tw_archive_clean = pd.merge(tw_archive_clean, pred_clean[['tweet_id','breed','prob', 'jpg_url', 'img_num']], on='tweet_id', how='left')

#### _**Test**_

In [85]:
tw_archive_clean.head(2)

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,dog_type,breed,prob,jpg_url,img_num
0,667550904950915073,NaN,NaN,2015-11-20 03:51:52 +0000,"<a href=""http://twitter.com"" rel=""nofollow"">Tw...",RT @dogratingrating: Exceptional talent. Origi...,6.675487e+17,4.296832e+09,2015-11-20 03:43:06 +0000,https://twitter.com/dogratingrating/status/667...,12,10,None,None,vizsla,0.000081,https://pbs.twimg.com/media/CUOb_gUUkAACXdS.jpg,1.0
1,667550882905632768,NaN,NaN,2015-11-20 03:51:47 +0000,"<a href=""http://twitter.com"" rel=""nofollow"">Tw...",RT @dogratingrating: Unoriginal idea. Blatant ...,6.675484e+17,4.296832e+09,2015-11-20 03:41:59 +0000,https://twitter.com/dogratingrating/status/667...,5,10,None,None,Unknown,0.000000,https://pbs.twimg.com/media/CUObvUJVEAAnYPF.jpg,1.0


Both datasets have merged correctly. But we can see that for example for the two first entries, the probability obtained from the neural net is very low. Once reviewed the images associated to the tweets it can be seen that these are screenshots of the mobile app and therefore the iamge is not correct for the net to predict correctly

### <u>Quality Issue 1: 'Incorrect datatypes (*_id, *timestamp)'<u>

#### __*Define*__

Convert _id columns into strings and timestamp columns into datetime format

#### __*Code*__

In [86]:
for each_col in tw_archive_clean.columns:
    if 'id' in each_col:
        tw_archive_clean[each_col] = tw_archive_clean[each_col].astype(str)

In [87]:
for each_col in tw_archive_clean.columns:
    if 'time' in each_col:
        tw_archive_clean[each_col] = pd.to_datetime(tw_archive_clean[each_col])

#### __*Test*__

In [88]:
tw_archive_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2356 entries, 0 to 2355
Data columns (total 18 columns):
 #   Column                      Non-Null Count  Dtype              
---  ------                      --------------  -----              
 0   tweet_id                    2356 non-null   object             
 1   in_reply_to_status_id       2356 non-null   object             
 2   in_reply_to_user_id         2356 non-null   object             
 3   timestamp                   2356 non-null   datetime64[ns, UTC]
 4   source                      2356 non-null   object             
 5   text                        2356 non-null   object             
 6   retweeted_status_id         2356 non-null   object             
 7   retweeted_status_user_id    2356 non-null   object             
 8   retweeted_status_timestamp  181 non-null    datetime64[ns, UTC]
 9   expanded_urls               2297 non-null   object             
 10  rating_numerator            2356 non-null   int64           

### <u>Quality Issue 2: 'retweet information not needed for this project'<u>

#### __*Define*__

Remove all rows with retweeted_status_id and in_reply_to_status_id then drop the columns related to retweet fields:
- in_reply_to_xxx= Nullable. If the represented Tweet is a reply, this field will contain the integer representation of the original Tweet’s xxx
- retweeted_status= Users can amplify the broadcast of Tweets authored by other users by retweeting . Retweets can be distinguished from typical Tweets by the existence of a retweeted_status attribute. This attribute contains a representation of the original Tweet that was retweeted. Note that retweets of retweets do not show representations of the intermediary retweet, but only the original Tweet. (Users can also unretweet a retweet they created by deleting their retweet.)
    
#### __*Code*__

In [89]:
tw_archive_clean = tw_archive_clean[tw_archive_clean.retweeted_status_id == 'nan']

In [90]:
tw_archive_clean = tw_archive_clean[tw_archive_clean.in_reply_to_status_id == 'nan']

In [91]:
tw_archive_clean = tw_archive_clean.drop(['retweeted_status_id','retweeted_status_user_id','retweeted_status_timestamp','in_reply_to_status_id','in_reply_to_user_id'],axis=1)

#### _**Test**_

In [92]:
tw_archive_clean.head()

,tweet_id,timestamp,source,text,expanded_urls,rating_numerator,rating_denominator,name,dog_type,breed,prob,jpg_url,img_num
2,667549055577362432,2015-11-20 03:44:31+00:00,"<a href=""http://twitter.com"" rel=""nofollow"">Tw...",Never seen dog like this. Breathes heavy. Tilt...,https://twitter.com/dog_rates/status/667549055...,1,10,None,None,Unknown,0.000000,https://pbs.twimg.com/media/CUOcVCwWsAERUKY.jpg,1.0
3,667546741521195010,2015-11-20 03:35:20+00:00,"<a href=""http://twitter.com"" rel=""nofollow"">Tw...",Here is George. George took a selfie of his ne...,https://twitter.com/dog_rates/status/667546741...,9,10,George,None,toy_poodle,0.787424,https://pbs.twimg.com/media/CUOaOWXWcAA0_Jy.jpg,1.0
4,667544320556335104,2015-11-20 03:25:43+00:00,"<a href=""http://twitter.com"" rel=""nofollow"">Tw...","This is Kial. Kial is either wearing a cape, w...",https://twitter.com/dog_rates/status/667544320...,10,10,Kial,None,Pomeranian,0.412893,https://pbs.twimg.com/media/CUOYBbbWIAAXQGU.jpg,1.0
5,667538891197542400,2015-11-20 03:04:08+00:00,"<a href=""http://twitter.com"" rel=""nofollow"">Tw...",This is a southwest Coriander named Klint. Hat...,https://twitter.com/dog_rates/status/667538891...,9,10,a,None,Yorkshire_terrier,0.618957,https://pbs.twimg.com/media/CUOTFZOW4AABsfW.jpg,1.0
6,667724302356258817,2015-11-20 15:20:54+00:00,"<a href=""http://twitter.com"" rel=""nofollow"">Tw...",What a dog to start the day with. Very calm. L...,https://twitter.com/dog_rates/status/667724302...,7,10,None,None,Unknown,0.000000,https://pbs.twimg.com/media/CUQ7tv3W4AA3KlI.jpg,1.0


### <u>Quality Issue 3: 'remove tweets without image'<u>

#### __*Define*__

Delete tweets where jpg_url is NaN
    
#### __*Code*__

In [93]:
tw_archive_clean = tw_archive_clean[tw_archive_clean.jpg_url.notnull()]

#### _**Test**_

In [94]:
tw_archive_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1971 entries, 2 to 2355
Data columns (total 13 columns):
 #   Column              Non-Null Count  Dtype              
---  ------              --------------  -----              
 0   tweet_id            1971 non-null   object             
 1   timestamp           1971 non-null   datetime64[ns, UTC]
 2   source              1971 non-null   object             
 3   text                1971 non-null   object             
 4   expanded_urls       1971 non-null   object             
 5   rating_numerator    1971 non-null   int64              
 6   rating_denominator  1971 non-null   int64              
 7   name                1971 non-null   object             
 8   dog_type            1971 non-null   object             
 9   breed               1971 non-null   object             
 10  prob                1971 non-null   float64            
 11  jpg_url             1971 non-null   object             
 12  img_num             1971 non-null 

In [95]:
tw_archive_clean[tw_archive_clean.jpg_url.isnull()]

,tweet_id,timestamp,source,text,expanded_urls,rating_numerator,rating_denominator,name,dog_type,breed,prob,jpg_url,img_num


### <u>Quality Issue 4: 'name columns contains words that are not names (ex. 'a')'<u>

#### __*Define*__

Review all dog "names" and modify the ones that are incorrect to "None".
    
#### __*Code*__

Looks like our main problem is that most incorrect names are 'a', 'an', 'the', etc. so mainly names that start with lower_case letters are wrong.

In [96]:
names=[]

def wrong_name(row):
    if row['name'][0].islower() == True:
        names.append('None')
    else:
        names.append(row['name'])
        
tw_archive_clean.apply(wrong_name, axis=1)

2       None
3       None
4       None
5       None
6       None
        ... 
2349    None
2351    None
2353    None
2354    None
2355    None
Length: 1971, dtype: object

In [97]:
tw_archive_clean['name'] = names

#### _**Test**_

In [98]:
tw_archive_clean['name'].unique()

array(['None', 'George', 'Kial', 'Frank', 'Dook', 'Hall', 'Philippe',
       'Naphaniel', 'Olive', 'Otis', 'Cleopatricia', 'Erik', 'Stu',
       'Tedrick', 'Calvin', 'Filup', 'Shaggy', 'Reese', 'Cupcake',
       'Clybe', 'Dave', 'Keet', 'Gabe', 'Klevin', 'Jeph', 'Jockson',
       'Carll', 'Pipsy', 'Bradlay', 'Biden', 'Fwed', 'Genevieve',
       'Joshwa', 'Timison', 'Clarence', 'Kenneth', 'Churlie', 'Jiminy',
       'Alfie', 'Rusty', 'Sophie', 'Jareld', 'Bisquick', 'Ron',
       'Skittles', 'Torque', 'DayZ', 'Jo', 'Kallie', 'Marvin', 'Spark',
       'Gòrdón', 'Josep', 'Lugan', 'Christoper', 'Jimothy', 'Kreggory',
       'Scout', 'Walter', 'Phineas', 'Monkey', 'Aja', 'Penny', 'Dante',
       'Nelly', 'Benedict', 'Venti', 'Goose', 'Nugget', 'Cash', 'Emmy',
       'Shadow', 'Jeffrey', 'Canela', 'Maya', 'Gerald', 'Mingus',
       'Waffles', 'Jimbo', 'Maisey', 'Derek', 'Ralphus', 'Zeke', 'Tilly',
       'Archie', 'Darla', 'Franklin', 'Jax', 'Zoey', 'Koda', 'Bruno',
       'Ted', 'Oliver', 'J

### <u>Tidiness Issue 3: 'tweepy dataframe should be merged into the "tw_archive" to consolidate all the info together.'<u>

#### __*Define*__

merge retweet and likes info into the tw_archive dataframe.
    
#### __*Code*__

In [103]:
tweepy_clean.tweet_id = tweepy_clean.tweet_id.astype(str)

In [104]:
tw_archive_clean = pd.merge(tw_archive_clean, tweepy_clean, on='tweet_id', how='left')

#### _**Test**_

In [105]:
tw_archive_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1971 entries, 0 to 1970
Data columns (total 15 columns):
 #   Column              Non-Null Count  Dtype              
---  ------              --------------  -----              
 0   tweet_id            1971 non-null   object             
 1   timestamp           1971 non-null   datetime64[ns, UTC]
 2   source              1971 non-null   object             
 3   text                1971 non-null   object             
 4   expanded_urls       1971 non-null   object             
 5   rating_numerator    1971 non-null   int64              
 6   rating_denominator  1971 non-null   int64              
 7   name                1971 non-null   object             
 8   dog_type            1971 non-null   object             
 9   breed               1971 non-null   object             
 10  prob                1971 non-null   float64            
 11  jpg_url             1971 non-null   object             
 12  img_num             1971 non-null 

### <u>Quality Issue 5: 'html tags in source text'<u>

#### __*Define*__

From source column extract the content from =  '> + content + <'
    
#### __*Code*__

In [109]:
tw_archive_clean.source.to_list()[1]

'<a href="http://twitter.com" rel="nofollow">Twitter Web Client</a>'

In [112]:
tw_archive_clean.source = tw_archive_clean.source.str.extract('>(.*)<', expand=True)

#### __*Test*__

In [117]:
tw_archive_clean.source.value_counts()

Twitter for iPhone    1932
Twitter Web Client      28
TweetDeck               11
Name: source, dtype: int64

### <u>Quality Issue 6: 'The missing data is encoded as "None"'<u>

#### __*Define*__

Replace the "None" for NaNs 
    
#### __*Code*__

In [120]:
tw_archive_clean.head(2)

,tweet_id,timestamp,source,text,expanded_urls,rating_numerator,rating_denominator,name,dog_type,breed,prob,jpg_url,img_num,favorites,retweet_count
0,667549055577362432,2015-11-20 03:44:31+00:00,Twitter Web Client,Never seen dog like this. Breathes heavy. Tilt...,https://twitter.com/dog_rates/status/667549055...,1,10,None,None,Unknown,0.000000,https://pbs.twimg.com/media/CUOcVCwWsAERUKY.jpg,1.0,NaN,NaN
1,667546741521195010,2015-11-20 03:35:20+00:00,Twitter Web Client,Here is George. George took a selfie of his ne...,https://twitter.com/dog_rates/status/667546741...,9,10,George,None,toy_poodle,0.787424,https://pbs.twimg.com/media/CUOaOWXWcAA0_Jy.jpg,1.0,NaN,NaN


The missing data in "name" or type of dog columns is encoded as 'None'.
some rating numerator and denominator have extreme max/min values
some tweets contain no information

Dog breed predictions with "(_)" separator.
Missing information as "tw_archive" file has 2356 rows and this one 2075. Nothing can be done as the predictions file is given and the neural net is not accessible.